### Car plate detection & recognation

In [1]:
import cv2
import numpy as np 
import imutils
import easyocr

C:\Users\user\anaconda3\anaconde file\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:

cap = cv2.VideoCapture("Cars_Plates.mp4")

reader = easyocr.Reader(['en'])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

   
    frame = cv2.resize(frame, (1000, 700)) # Resize the frame
    
    
    roi = frame[-200:, 200:-200] # Define  (ROI)

    
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)# Convert ROI to grayscale
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)  # Apply bilateral filter    
    edged = cv2.Canny(bfilter, 30, 200)# Apply Canny

    # Find contours 
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:100]

    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 5, True)
        if len(approx) == 4:
            location = approx
            break

    # Extract the area of the text
    if location is not None:
        mask = np.zeros(gray.shape, np.uint8)
        new_image = cv2.drawContours(mask, [location], -1, 255, -1)
        new_image = cv2.bitwise_and(roi, roi, mask=mask)

        (x, y) = np.where(mask == 255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        cropped_image = gray[x1:x2 + 1, y1:y2 + 1]

        
        result = reader.readtext(cropped_image)# Read text using EasyOCR
        if result:
            text = result[0][-2]
            font = cv2.FONT_HERSHEY_SIMPLEX
            #res = cv2.putText(roi, text=text, org=(location[0][0][0], location[1][0][1] + 60), fontFace=font,
                              #fontScale=1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
            res = cv2.rectangle(roi, tuple(location[0][0]), tuple(location[2][0]), (0, 255, 0), 3)

    # Display the frame with ROI
    cv2.rectangle(frame,(800,1000),(200,500),(255,255,255),1)
    cv2.putText(frame, text='ROI', org=(465,490), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2, lineType=cv2.LINE_AA)

    
    cv2.imshow('frame', frame)
#     cv2.imshow('roi', roi)
#     cv2.imshow('edged', edged)
#     cv2.imshow('bfilter', bfilter)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


### Wish u luck
### Esraa Meslam